# Two-Stream Instability with PlasmaTorch

## Introduction
In this tutorial, we demonstrate how **PlasmaTorch** can simulate a classical **two-stream instability**. We initialize two counter-streaming electron beams in a periodic domain, with a stationary (or effectively fixed) ion background. The relative motion of the two beams sets up electric fields that can grow exponentially due to the **two-stream instability**.

### Physical Background
In the **two-stream instability**, two populations of electrons with different drift velocities interact via electrostatic perturbations. In the simplest 1D analysis, the dispersion relation of electrostatic waves can exhibit modes with exponential growth. If the relative drift between the beams is high enough, small fluctuations in density and electric field are amplified, leading to a rapid energy exchange.

In more detail, for electrons of equal density $n_0$, masses $m$, charges $-e$ (in standard SI units), and drift velocities $\pm v_0$, the condition for instability typically emerges around wave numbers satisfying:
$$ k v_0 \approx \omega_{pe}, $$
where $$ \omega_{pe} = \sqrt{\frac{n_0 \, e^2}{\varepsilon_0 \, m}}. $$ is the electron plasma frequency.

In this tutorial, we use a 2D domain with periodic boundary conditions, create a large number of electrons in two beams with equal and opposite velocities, and add a set of ions to act as a neutralizing background. We then observe how the system evolves, with the possibility of seeing kinetic energy transferred into electric-field fluctuations characteristic of the two-stream instability.

## Cell 1: Imports and Setup
We:
- Reload modules with `%autoreload`.
- Import standard libraries like **torch**, **numpy**, **matplotlib**, and **tqdm**.
- Import **PlasmaTorch** modules.
- Set the device (`cuda` if available, otherwise `cpu`).
- Initialize the random seed for reproducibility.

In [ ]:
%load_ext autoreload
%autoreload 2

import torch
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import torch.autograd.profiler as profiler
from scipy.constants import *
import sys
sys.path.append("../")
sys.path.append("../plasmatorch")

from plasmatorch.simulator import *
from plasmatorch.utils import *
from plasmatorch.deposition import *
from plasmatorch.fields import *
from plasmatorch.helper import *

device = "cuda" if torch.cuda.is_available() else "cpu"
set_seed(2304)

## Cell 2: Defining the Simulation Parameters
Here, we:
- Create a 2D domain of size $L_x = L_y = 1.0$ with a $64 \times 64$ grid.
- Use a time step of $10^{-6}$ and run up to $t = 10^{-3}$.
- Name the simulation `"test"`.
- Use periodic boundary conditions.
- Choose to save the simulation data at every time step (`save_frequency=1`).
- Use **Gauss-Seidel** for solving the Poisson equation.

Then we set up a quasi-neutral background with ions (with large mass, or equivalently, you could treat them as stationary), and create **two** electron beams with opposite velocities in the $x$-direction.

In [ ]:
Lx, Ly = 1.0, 1.0
Nx, Ny = 64, 64
dt = 1e-6
simulation_time = 1e-3
simulation_name = "test"
backend = "cpu"

sim = PlasmaTorch(
    simulation_name=simulation_name,
    Lx=Lx,
    Ly=Ly,
    Nx=Nx,
    Ny=Ny,
    simulation_time=simulation_time,
    dt=dt,
    boundaries_type="periodic",
    save_frequency=1,
    backend= backend,
)

sim.poisson_method = "gauss_seidel"

# Create ions as a stationary or massive background
# We choose 5000 ions with zero initial velocity and large mass
num_ioni = 5000
sim.create_new_specie(
    name="ioni",
    num_particles=num_ioni,
    temperature=0.0,
    distribution="zero",
    mass=1836.0,  # large mass to mimic protons
    charge=1.0,
    disposition="random"
)

# Create two beams of electrons
num_elettroni = 5000

# Beam 1 with +vx
sim.create_new_specie(
    name="electrons_beam1",
    num_particles=num_elettroni,
    temperature=0.0,
    distribution="zero",
    mass=1.0,
    charge=-1.0,
    disposition="random"
)
# Assign a positive velocity in x
is_beam1 = (sim.particles_specie == sim.name_register.tolist().index("electrons_beam1"))
sim.particles_velocity[is_beam1, 0] = 0.2  # velocity in +x

# Beam 2 with -vx
sim.create_new_specie(
    name="electrons_beam2",
    num_particles=num_elettroni,
    temperature=0.0,
    distribution="zero",
    mass=1.0,
    charge=-1.0,
    disposition="random"
)
# Assign a negative velocity in x
is_beam2 = (sim.particles_specie == sim.name_register.tolist().index("electrons_beam2"))
sim.particles_velocity[is_beam2, 0] = -0.2  # velocity in -x

# Run the simulation
sim.simulate()

## Cell 3: Loading the Simulation
If we saved the simulation data, we can now load it for post-processing, analysis, or further diagnostics without needing to re-run the simulation.

In [ ]:
simulation = load_simulation(simulation_name, device= backend)

## Cell 4: Interactive Visualization
By using `%matplotlib widget` and the **`dynamic_slider`** function, we can step through the simulation time steps to see how the beams evolve and potentially create oscillations or filamentation that are characteristic of the two-stream instability.
In 2D, you might see structures forming in the $x$-$y$ plane as the two beams interact.

In [ ]:
%matplotlib widget

dynamic_slider(simulation, Lx, Ly)

## Cell 5: Energy Analysis
Let's look at the evolution of kinetic, potential, and total mechanical energy over the time steps. One hallmark of the two-stream instability is that electrostatic energy can grow at the expense of the beams' kinetic energy.


In [ ]:
plt.close()
plt.figure()
plt.plot(simulation['kinetic_energy_chronology'], label="Kinetic Energy")
plt.plot(simulation['potential_energy_chronology'], label="Potential Energy")
plt.plot(simulation['mechanic_energy_chronology'], label="Mechanical Energy")
plt.legend()
plt.show()

## Conclusion
In this tutorial, we set up two counter-streaming electron beams in a periodic 2D domain with ions acting as a neutralizing background. **PlasmaTorch** handles the self-consistent electric field solution, allowing us to observe whether an instability develops. In classical plasma theory, this configuration is subject to the **two-stream (two-flow) instability**, which can lead to the exponential growth of electric field modes that couple to the beams. By examining the particle distribution and field energy over time, one can verify the onset and evolution of this well-known plasma instability.